## classifying the yelp reviews

In [19]:
#imports 
import re
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer ,TfidfVectorizer
from sklearn.model_selection import train_test_split,GridSearchCV
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import accuracy_score,confusion_matrix 
from sklearn.linear_model import SGDClassifier,LogisticRegression
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.ensemble import RandomForestClassifier ,VotingClassifier
from sklearn.naive_bayes import MultinomialNB

In [9]:
# reading the data 
yelp = pd.read_csv('yelp.csv')
# selecting the wanted data 
yelp_best_worst = yelp[(yelp.stars==5) | (yelp.stars==1)]
# assigning the feature matrix to the text of the review
X = yelp_best_worst.text
# assigning the target array which is binary 1 star and 5
y = yelp_best_worst.stars
# mapping the values of the array y to new values 1, 0 
y = y.map({5:1,1:0})

In [10]:
# splitting the data to training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [11]:
# list of the stop words to remove from the review because it doesn't add to our model and it it's not useful since it has no way of telling if the review is positive of negative like [is, are, the ]
stop_words = stopwords.words('english')
#count vectorization is basically turning our one column matrix into a sparse matrix with the number of columns equal to the number of tokens 
vect = CountVectorizer(stop_words=stop_words,min_df=2,ngram_range=(1,2))
#transforming the data 
X_train = vect.fit_transform(X_train)
X_test = vect.transform(X_test)

In [12]:
nb = MultinomialNB()
rf =RandomForestClassifier(n_estimators=100)
sg = SGDClassifier()
lr = LogisticRegression()

In [13]:
def estim_score(estimators,accuracy=False,confusion=True):
    """loop over the estimators and prints the score and confusion matrix"""
    for i in estimators:
        i.fit(X_train,y_train);
        y_true = i.predict(X_test)
        if accuracy == True:
            print('the accuract of the  estimator is {}'.format(accuracy_score(y_test,y_true)))
        if confusion == True:
            print(confusion_matrix(y_test,y_true))

In [44]:
estimators_list = [lr,rf, nb, sg]
estim_score(estimators_list,accuracy=True)

the accuract of the  estimator is 0.9324853228962818
[[137  47]
 [ 22 816]]
the accuract of the  estimator is 0.901174168297456
[[ 92  92]
 [  9 829]]
the accuract of the  estimator is 0.9285714285714286
[[134  50]
 [ 23 815]]
the accuract of the  estimator is 0.9148727984344422
[[141  43]
 [ 44 794]]


####  traing the model on the all of the data

In [58]:
  
X = np.concatenate((X_train.toarray(),X_test.toarray()),axis=0)

In [60]:
y = np.concatenate((y_train,y_test),axis=0)

In [61]:
lr.fit(X,y)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [94]:
rev = '''
I have lived a block from this Domino's for over two and a half years. It's had several ups and downs over that time, a complete shakeup of its staff, and yet -- thanks to convenience and an intense work/school schedule that keeps me out of the house 12 or more hours a day, seven days a week -- I kept coming back. Today, however, I bid a not-remotely-fond farewell to this neighborhood staple.

It's Domino's. I know. It's not Absinthe or Quince or the Ritz. I get that, and I accept that, for all that Domino's is and is not. It's simple and you know exactly what you're going to get. Except when you don't, as was the case this afternoon. 

I ordered online as usual, keeping it simple: basic, customized pasta and a 20-ounce Diet Coke. (You know, for health.) The pasta (burned on top, at-best-lukewarm in the middle) was pretty much what I expected, despite a smaller-than-usual serving. I could handle that, but for some reason, a minor sleight boiled over today: they were out of Diet Coke and I took home a root beer. It wasn't so much the exchanged/replaced item as the -- as of now, completely standard -- utter lack of care exhibited by the entire staff. I've worked in retail and I know, deeply and truly, how awful it can be. People are the worst, are belligerent and prone to lashing out at service staff (wrong and profoundly vile behavior, always, full stop). But I am not that person and I was not that person. I simply asked if they had anything with caffeine in it; the purpose of the Diet Coke, after an already-10-hour-day, looking ahead to a very late evening of work. No dice. Worse, no apology, no contrition of any kind, no effort to make amends. The only 20-ounce beverage they had (and, as of this writing, have) was root beer. Fine. But don't just shove it into my chest and say "OK see you." No. It is not OK and you will not be seeing me again, lest I walk by scowling at you.

This is the culmination of a year of increasing showings of complete staff incompetence and disregard for customers. Like I said, Domino's probably sucks to work at, infinitely more than 90% of any other retail experience I can imagine. I probably sucked a lot when the last staff worked there, before they were systematically replaced. But that previous staff, now nearly two years back, was composed of courteous, kind people who, despite slip-ups and omissions, apologized (earnestly!) and always sought to send customers happily on their way -- not just out the door. That every interaction over the last year, dwindling as they are from once every few weeks to once every few months, now, begins with no one acknowledging my presence in the store, backs to me, not listening when I give my name and having to repeat myself numerous times, all the while watching the staff blatantly disregard phone calls while standing around doing nothing (certainly not helping customers!) has grated beyond belief. I know, I'm beyond belief that I'm even taking the time to write this, but I couldn't fathom unloading this on people who (despite incompetence, casual indifference, and a complete disregard for basic conventions of etiquette) have a job I do not envy in the slightest. 

Long story short, whatever managerial overhaul this location went through a year or two ago has clearly just accelerated the decline of an already-pitiable fast food go-to. If you're famished and in a fix, you might as well go to the adjacent hardware store and pick up some Funyuns and a soda -- better quality food, better price, and better service, guaranteed.
'''

In [95]:
lr.predict(vect.transform(pd.Series([rev])))

array([0])